# Исследование надежности заемщиков

## Изучение общей информации

In [1]:
import pandas as pd

try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

In [2]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

In [4]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

 В двух столбцах есть пропущенные значения. Один из них — `days_employed`. Другой столбец с пропущенными значениями — `total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца `income_type`. Например, у человека с типом занятости `сотрудник` пропуск в столбце `total_income` должен быть заполнен медианным доходом среди всех записей с тем же типом.

In [5]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. таким артефактом будет отрицательное количество дней трудового стажа в столбце `days_employed`.

In [6]:
data['days_employed'] = data['days_employed'].abs()

In [7]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, оставлю их как есть. Тем более этот столбец не понадобится для исследования.

In [10]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

In [11]:
data = data[(data['children'] != -1) & (data['children'] != 20)] # удаляю аномальные значения

In [12]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

### Удаление пропусков (продолжение)

In [13]:
# Заполню пропуски в столбце days_employed медианными значениями по каждого типа занятости income_type
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

In [14]:
data.isna().sum() # Убеждаюсь, что пропуски заполнены

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

In [15]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

In [17]:
data['education'] = data['education'].str.lower() #В этом столбце есть одни и те же значения, 
#но записанные по-разному: с использованием заглавных и строчных букв. 

In [18]:
data.duplicated().sum()

71

In [19]:
data = data.drop_duplicates()

### Категоризация данных

In [20]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [21]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

In [22]:
data['purpose'].unique() #перечень уникальных целей взятия кредита из столбца purpose.

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [23]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [24]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

###  Исследование данных

#### Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [25]:
data.info() # посмотрим данные о таблице

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21331 entries, 0 to 21524
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   children               21331 non-null  int64  
 1   days_employed          21331 non-null  float64
 2   dob_years              21331 non-null  int64  
 3   education              21331 non-null  object 
 4   education_id           21331 non-null  int64  
 5   family_status          21331 non-null  object 
 6   family_status_id       21331 non-null  int64  
 7   gender                 21331 non-null  object 
 8   income_type            21331 non-null  object 
 9   debt                   21331 non-null  int64  
 10  total_income           21331 non-null  int32  
 11  purpose                21331 non-null  object 
 12  total_income_category  21331 non-null  object 
 13  purpose_category       21331 non-null  object 
dtypes: float64(1), int32(1), int64(5), object(7)
memory us

In [26]:
def proportion(pdSerises): # функция, которая будет находить отношение значений в % с округлением до 2 знаков
    return str(round((pdSerises.sum() / pdSerises.count()) * 100, 2)) + '%' 

Сформирую сводную таблицу с количеством задержек по кредиту, количеством заемщиков и процентом задержек в зависимости от количества детей

In [27]:
data_children = data.pivot_table(index=['children'], values='debt', aggfunc=['sum','count', proportion]) 
data_children.sort_values(by=('proportion', 'debt')) #вывожу таблицу на экран отсортировав по проценту невозврата

,sum,count,proportion
,debt,debt,debt
children,,,
5,0,9,0.0%
0,1063,14091,7.54%
3,27,330,8.18%
1,444,4808,9.23%
2,194,2052,9.45%
4,4,41,9.76%


**Вывод:**  Зависимость между количеством детей и возвратом кредита в срок существует. Процент задержки по возврату кредитов среди заемщиков растет с увеличением количества детей у заемщика, самый низкий у заемщиков без детей (7,54%), у заемщиков с 1 ребенком резко вырастает почти на 2%, с двумя еще немного возрастает, при этом для заемщиков с тремя детьми он падает больше чем на 1% отноительно заемщиков с 2 детьми, а для заемщиков с 4 детьми снова резко вырастает, что странно для заемщиков с 5 детьми равен нулю. Хотя выборка заемщиков с 5 детьми составляет всего 9 человек и вероятно она не репрезентативна, так как кредит в среднем задерживает 1 из 11-12 заемщиков с детьми.

#### Есть ли зависимость между семейным положением и возвратом кредита в срок?

Сформирую сводную таблицу с количеством задержек по кредиту, количеством заемщиков и процентом задержек в зависимости от семейного статуса

In [28]:
data_total_income = data.pivot_table(index=['total_income_category'], values='debt', aggfunc=['sum','count', proportion]) 
data_total_income.sort_values(by=('proportion', 'debt')) #вывожу таблицу на экран отсортировав по проценту невозврата

,sum,count,proportion
,debt,debt,debt
total_income_category,,,
D,21,349,6.02%
B,354,5014,7.06%
A,2,25,8.0%
C,1353,15921,8.5%
E,2,22,9.09%


**Вывод:** Из сформированной разбивки на категории видно, что самое большое количество просрочек допускают заемщики с самым низким доходом (менее 30 тыс), с доходом в диапазоне от 50 до 200 тысяч рублей и с самым высоким доходом (более 1 млн). Заемщиков с доходом менее 30 тысяч и более 1 млн в выборке очень мало и возможно выводы некорректны. Аккуратнее всего платят кредиты граждане с ежемесячными доходами от 50 до 200 тысяч и от 200 до 1 млн. 

####  Как разные цели кредита влияют на его возврат в срок?

Сформирую сводную таблицу с количеством задержек по кредиту, количеством заемщиков и процентом задержек в зависимости от целей кредитования

In [29]:
data_purpose_category = data.pivot_table(index=['purpose_category'], values='debt', aggfunc=['sum','count', proportion]) #
data_purpose_category.sort_values(by=('proportion', 'debt')) #вывожу таблицу на экран отсортировав по проценту невозврата

,sum,count,proportion
,debt,debt,debt
purpose_category,,,
операции с недвижимостью,780,10751,7.26%
проведение свадьбы,183,2313,7.91%
получение образования,369,3988,9.25%
операции с автомобилем,400,4279,9.35%


**Вывод:** Лучше всего кредиты возвращают заемщики бравшие их на операции с недвижимостью (7,26%), чуть хуже возвращают кредиты на свадьбу (7,91%), заметно хуже возвращают кредиты на образование и автомобиль 9,25 и 9,35% соответственно).

### Общий вывод.

Вероятность возврата кредита в срок зависит от множества факторов. Самая низкая вероятность просрочки у вдовца/вдовы без детей с ежемесячным доходом от 200 тысяч до 1 млн берущего кредит на недвижимость. Самая высокая у заемщика с 4 детьми не состоявшего в браке с доходом до 30 тысяч берущего кредит на автомобиль. Вероятность просрочки примерно одинаковая по всем параметрам и лежит в диапазоне от 6 до 9.8%. При этом стоит отметить, что размер выборки для заемщиков с 5 детьми очень маленький для того что бы можно было делать выводы. Так же небольшими получились выборки для заемщиков с доходами от 1 млн и до 30 тысяч, возможно стоит сделать другую разбивку по доходам, пока неочевидно какие должны быть параметры для выборки граничных условий диапазонов. Так же можно увидеть что наибольший разброс между минимальным и максимальным значением просрочки принадлежит к категории с семейным положением 3,14%, а минимальный к категории целей кредита 2,09%.